# RG374 FACS analysis IFNAR fl LysMcre CpG

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
                         
            # Data 
            library(tidyverse), 
            library(data.table), 
            library(reactable), 

            # Plotting 
            library(ComplexHeatmap), 
            library(patchwork), 
            library(cowplot), 
            library(ggrepel)

        )
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Functions 

In [ ]:
scale_label <- function(x, y) {
    
    max_y <- max(x[[y]])
    float_y <- attr(regexpr("(?<=\\.)0+", max_y, perl = TRUE), "match.length")
    
    if(max_y>=1) {
    
        x <- x
        ylab="10^6~x~cells/spleen"
        
        return(list(mat=x, ylab=ylab))

    } else if (float_y==-1) {
        
        x[[y]] <- x[[y]]*10
        ylab <- paste0("10^5~x~cells/spleen")

        return(list(mat=x, ylab=ylab)) 
    
    } else {

        exp_scale <- 1+float_y

        x[[y]] <- x[[y]]*10^exp_scale
        ylab <- paste0("10^", 6-exp_scale, "~x~cells/spleen")

        return(list(mat=x, ylab=ylab))

    }
        
}

In [ ]:
bar_plot <- function(mat, x, y, xlab="", ylab="", ggtitle="", scale_y=FALSE, parse_y=FALSE, expand_y=NULL) {
    
    if(scale_y) {
        
        res <- scale_label(mat, deparse(substitute(y)))
        mat <- res[[1]]
        ylab <- res[[2]]

    }
    
    p <- ggplot(mat, aes(x={{x}}, y={{y}}, color=genotype, fill=genotype)) + 
    
        geom_bar(stat="summary", width=0.8, color="black", size=0.1) + 
        geom_jitter(shape=21, stroke=0.1, size=1.0, show.legend=FALSE, color="black") + 
        xlab(xlab) + ylab(ylab) + ggtitle(ggtitle) + 
        scale_color_manual(values=color$genotype) + 
        scale_fill_manual(values=color$genotype) + 
        theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
        guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))
    
    if(parse_y) {
        
        p <- p + ylab(parse(text=ylab))
        
    }


    if(!is.null(expand_y)) {

         p <- p + ylim(0, (max(ggplot_build(p)$data[[1]]$y) + expand_y))
    }
    
    return(p)
    
}

# Import data 

In [ ]:
sheet_names <- openxlsx::getSheetNames("data/RG374_ifnar_fl_lysmcre_cpg/facs/spleen//RG374_facs_spleen_data.xlsx")

In [ ]:
data <- lapply(sheet_names, function(x) openxlsx::read.xlsx("data/RG374_ifnar_fl_lysmcre_cpg/facs/spleen//RG374_facs_spleen_data.xlsx", sheet=x))
names(data) <- sheet_names

# Set color

In [ ]:
color$genotype <- c("+/+"="#FFFFFF", "cre/+"="#998989")

## Spleen weight 

In [ ]:
mat <- data[["spleen_overview"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample, -viable_cells) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>%
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6")))

In [ ]:
options(repr.plot.width=5, repr.plot.height=3.5)

bar_plot(mat, genotype, spleen_weight_mg, ylab="Spleen weight [mg]")

In [ ]:
pdf("result/figures/dev/bp_spleen_weigth.pdf", width=2, height=4)

gridExtra::grid.arrange(

    (bar_plot(mat, genotype, spleen_weight_mg, ylab="Spleen weight [mg]", expand_y=100) + theme_global_set(4)) %>% egg::set_panel_size(., width=unit(6*0.3, "cm"), height=unit(2.0, "cm")), ncol=1
        
)

dev.off()

## Viable cells 

In [ ]:
mat <- data[["spleen_overview"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample, -spleen_weight_mg) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(viable_cells=viable_cells/10^4)

In [ ]:
options(repr.plot.width=5, repr.plot.height=3.5)

bar_plot(mat, genotype, viable_cells, ylab="10^6~viable~cells/spleen", parse_y=TRUE)

## CD44 SSC gating 

In [ ]:
mat <- data[["cd44_ssc_gating"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cell_type", value.name=c("ratio"))  

In [ ]:
options(repr.plot.width=10, repr.plot.height=3.5)

bar_plot(mat, cell_type, ratio, ylab="10^6~viable~cells/spleen", parse_y=TRUE) + facet_grid(~time_point)

# Relative cell numbers 

In [ ]:
mat <- data[["relative"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cell_type", value.name=c("ratio"))  

In [ ]:
bar_plot_1 <- lapply(split(mat, f=mat$cell_type), function(x) bar_plot(x, genotype, ratio, ylab="Ratio [%]", ggtitle=x$cell_type[1]) + theme_global_set(4))

In [ ]:
options(repr.plot.width=5*3, repr.plot.height=3*3.5)

ggpubr::ggarrange(plotlist=bar_plot_1, ncol=5, nrow=3, common.legend=TRUE, legend="bottom")

In [ ]:
pdf("result/figures/dev/bp_facs.pdf", width=2, height=4)

gridExtra::grid.arrange(

    bar_plot_1[[1]] %>% egg::set_panel_size(., width=unit(6*0.3, "cm"), height=unit(2.0, "cm")), ncol=1
        
)

dev.off()

## Absolut numbers 

In [ ]:
mat <- data[["absolut"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cell_type", value.name=c("count"))

In [ ]:
bar_plot_1 <- lapply(split(mat, f=mat$cell_type), function(x) bar_plot(x, time_point, count, scale_y=TRUE, parse_y=TRUE, ggtitle=x$cell_type[1]))

In [ ]:
options(repr.plot.width=5*3, repr.plot.height=3*3.5)

ggpubr::ggarrange(plotlist=bar_plot_1, ncol=5, nrow=3, common.legend=TRUE, legend="bottom")